In [1]:
import jieba
from gensim import corpora, similarities, models
from openpyxl import load_workbook
import random
import io

workbook = load_workbook(u'moli_for_cross_domain.xlsx')
booksheet = workbook.active
rows = booksheet.rows
columns = booksheet.columns
line_list = []
for row in rows:
    line = [col.value for col in row]
    line_list.append(line)

dict = {}
cases = []
case_subintents = []
for line in line_list[1:]:
    cases.append(line[1])
    case_subintents.append(line[2])
    supintent = line[2]
    if supintent not in dict.keys():
        dict[supintent] = []
    dict[supintent].append(line[1])
subintent_set = [k for k in dict.keys()]
sentances = cases + case_subintents
test_sents = subintent_set
text = [[word for word in jieba.cut(sentance)]for sentance in sentances]  # 1.把每个句子分词
dictionary  = corpora.Dictionary(text)  # 2.把每个词语建立索引，得到索引字典
# print('dictionary=', dictionary)
# for idx,word in dictionary.items():
#     print(idx, word,end="\t")
# print()

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.103 seconds.
Prefix dict has been built successfully.


In [2]:
corpus = [dictionary.doc2bow(word_list) for word_list in text]  # 3.对每句话的每个词语进行词频统计,得到词频统计过后的语料corpus
# print("[dictionary.doc2bow(word_list) for word_list in text]")
# for word_list in text:
#     print('\t',word_list, end="\t")
#     print(dictionary.doc2bow(word_list))

In [3]:
model = models.TfidfModel(corpus)  # 4. corpus输入到TFIDF模型计算，model保存着有每句话中每个词语的tfidf值
tfidf = model[corpus]  #  保存着每句话中每个词语的tfidf值
# print('tfidf=',tfidf)
# for ele in tfidf:  
#     print('\t',ele)

In [5]:
for test_sent in test_sents:
    test_word_list = [word for word in jieba.cut(test_sent)]
    print('test_word_list=',test_word_list)
    test_word_freq_count = dictionary.doc2bow(test_word_list)
    test_tfidf = model[test_word_freq_count]
    print(test_word_freq_count)
    test_tfidf_score = [t[1] for t in test_tfidf]
    print('test_tfidf=', test_tfidf_score)

test_word_list= ['unable', ' ', 'to', ' ', 'sync']
[(0, 2), (13, 1), (29, 1), (690, 1)]
test_tfidf= [0.002714847081987764, 0.21071565623838115, 0.6487512170891342, 0.7312410000449351]
test_word_list= ['cannot', ' ', 'hear', ' ', 'ringtone']
[(0, 2), (34, 1), (86, 1), (471, 1)]
test_tfidf= [0.002432061985803844, 0.594264740853218, 0.390150267089964, 0.703296716857675]
test_word_list= ['cannot', ' ', 'be', ' ', 'heard']
[(0, 2), (86, 1), (298, 1), (1818, 1)]
test_tfidf= [0.0023434810377939304, 0.3759401520818437, 0.4984831491203842, 0.7811389506306355]
test_word_list= ['will', ' ', 'not', ' ', 'charge']
[(0, 2), (11, 1), (45, 1), (84, 1)]
test_tfidf= [0.00354079791315436, 0.39146815200705337, 0.7171494827941597, 0.5765732980657274]
test_word_list= ['cannot', ' ', 'connect', ' ', 'to', ' ', 'pc', ' ', 'via', ' ', 'usb']
[(0, 5), (13, 1), (86, 1), (92, 1), (219, 1), (334, 1), (504, 1)]
test_tfidf= [0.0048337330510643236, 0.15007007042038056, 0.3101700946640982, 0.48385867516197717, 0.47575